# Example
The below is an example of how to use the various reaction representations provided in this github repo (SLATM_r, SLATM_d, BoB_r, BoB_rp, SOAP_d, etc, as well as our own B2R2) for the SN2 dataset.

The four datasets (SN2, Grambow, hfb and e.e.) referenced in the QML for reactions paper are available with equivalent functions.

For your own datasets, see the other example notebook.

In [1]:
import numpy as np

In [2]:
from src.reaction_reps import QML, DScribe, B2R2
from src.learning import opt_hyperparams, learning_curve

In [3]:
import matplotlib.pyplot as plt

First generate representations from B2R2 series:

In [4]:
b2r2 = B2R2()
b2r2.get_ee_data()

In [5]:
b2r2_l = b2r2.get_b2r2_l(Rcut=4)

Now generate representations from qml python package (CM, BoB, SLATM, FCHL).

In [6]:
qml = QML()
qml.get_ee_data()

In [7]:
energies = qml.energies

In [ ]:
slatm_r, slatm_p, slatm_d = qml.get_SLATM()

In [ ]:
cm_r, cm_p, cm_rp = qml.get_CM()

In [ ]:
bob_r, bob_p, bob_rp = qml.get_BoB()

In [ ]:
# make sure you've installed develop branch of qml for fchl19

In [16]:
fchl_r, fchl_p, fchl_d = qml.get_FCHL19()

AttributeError: module 'qml.representations' has no attribute 'generate_fchl_acsf'

Now generate representations from dscribe package (SOAP)

In [ ]:
ds = DScribe()
ds.get_ee_data()

In [ ]:
soap_r, soap_p, soap_d = ds.get_SOAP()

In [13]:
# find hyperparams in SI (or use opt_hyperparams function)

In [ ]:
train_sizes, mae_slatm_d, std_slatm_d = learning_curve(slatm_d, energies, CV=10, sigma=100, l2reg=1e-4)

In [ ]:
train_sizes, mae_soap_d, std_soap_d = learning_curve(soap_d, energies, CV=10, sigma=1, l2reg=1e-4)

In [ ]:
train_sizes, mae_fchl_d, std_fchl_d = learning_curve(fchl_d, energies, CV=10, sigma=1000, l2reg=1e-8)

In [ ]:
train_sizes, mae_bob_rp, std_bob_rp = learning_curve(bob_rp, energies, CV=10, sigma=1000, l2reg=1e-6)

In [ ]:
train_sizes, mae_cm_rp, std_cm_rp = learning_curve(cm_rp, energies, CV=10, sigma=1000, l2reg=1e-4)

In [ ]:
train_sizes, mae_b2r2_l, std_b2r2_l = learning_curve(b2r2_l, energies, CV=10, sigma=1e4, l2reg=1e-6)

In [ ]:
def polyfit(train_sizes, maes):
    m, c = np.polyfit(np.log(train_sizes), np.log(maes), 1)
    polyfit = np.exp(m * np.log(train_sizes) + c)
    return polyfit

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_xscale('log')
ax.set_yscale('log')

ax.scatter(train_sizes, mae_slatm_d, color='purple', label='SLATM$_d$')
ax.plot(train_sizes, polyfit(train_sizes, mae_slatm_d), color='purple')

ax.scatter(train_sizes, mae_soap_d, color='magenta', label='SOAP$_d$')
ax.plot(train_sizes, polyfit(train_sizes, mae_soap_d), color='magenta')

#ax.scatter(train_sizes, mae_fchl_d, color='red', label='FCHL$_d$')
#ax.plot(train_sizes, polyfit(train_sizes, mae_fchl_d), color='red')

ax.scatter(train_sizes, mae_bob_rp, color='orange', label='BoB$_{rp}$')
ax.plot(train_sizes, polyfit(train_sizes, mae_bob_rp), color='orange')

ax.scatter(train_sizes, mae_cm_rp, color='gold', label='CM$_{rp}$')
ax.plot(train_sizes, polyfit(train_sizes, mae_cm_rp), color='gold')

ax.scatter(train_sizes, mae_b2r2_l, color='turquoise', marker='*', label='$B^2R^2_l$')
ax.plot(train_sizes, polyfit(train_sizes, mae_b2r2_l), color='turquoise')

ax.set_xlabel("Number of training instances")
ax.set_ylabel("MAE $e.e.$ / kcal/mol")
plt.legend()
plt.tight_layout()
plt.show()